<a href="https://colab.research.google.com/github/8sheeta8/2025_Capstone_DDoS-Botnet/blob/main/DDoS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# 정상 데이터
normal = np.load('/content/normal_combined.npy')
print("✅ normal shape:", normal.shape)

# DDoS 공격 데이터
http = np.load('/content/http_flood.npy')
icmp = np.load('/content/icmp_flood.npy')
tcp_ack = np.load('/content/tcp_ack.npy')
tcp_other = np.load('/content/tcp_other.npy')
udp_other = np.load('/content/udp_other.npy')

# 각각 shape 확인
print("📂 http_flood:", http.shape)
print("📂 icmp_flood:", icmp.shape)
print("📂 tcp_ack:", tcp_ack.shape)
print("📂 tcp_other:", tcp_other.shape)
print("📂 udp_other:", udp_other.shape)


✅ normal shape: (106561, 1024)
📂 http_flood: (378, 500)
📂 icmp_flood: (702, 500)
📂 tcp_ack: (5389, 500)
📂 tcp_other: (5389, 500)
📂 udp_other: (184, 500)


In [3]:


def pad_array(arr, target_len=1024):
    if arr.shape[1] >= target_len:
        return arr[:, :target_len]
    else:
        pad_width = target_len - arr.shape[1]
        return np.pad(arr, ((0, 0), (0, pad_width)), mode='constant')

# 모든 데이터 길이 1024로 맞춤
normal = pad_array(normal)
http = pad_array(http)
icmp = pad_array(icmp)
tcp_ack = pad_array(tcp_ack)
tcp_other = pad_array(tcp_other)
udp_other = pad_array(udp_other)

ddos = np.concatenate([http, icmp, tcp_ack, tcp_other, udp_other], axis=0)
X = np.concatenate([normal, ddos], axis=0)

# 입력과 라벨 생성
y = np.array([0]*len(normal) + [1]*len(ddos))

print("✅ 전체 데이터 shape:", X.shape)
print("✅ 라벨 shape:", y.shape)


✅ 전체 데이터 shape: (118603, 1024)
✅ 라벨 shape: (118603,)


In [4]:
# CNN 입력을 위한 reshape (샘플 수, 특징 수, 채널 수)
X = X.reshape((X.shape[0], X.shape[1], 1))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

print("🎯 학습 데이터:", X_train.shape, y_train.shape)
print("🎯 테스트 데이터:", X_test.shape, y_test.shape)


🎯 학습 데이터: (59301, 1024, 1) (59301,)
🎯 테스트 데이터: (59302, 1024, 1) (59302,)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Conv1D(64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # 이진 분류
])

model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1022, 32)       │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1022, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 511, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 511, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 509, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 509, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 254, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 254, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16256)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,080,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,087,745 (7.96 MB)

 Trainable params: 2,087,553 (7.96 MB)

 Non-trainable params: 192 (768.00 B)

In [6]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/10
927/927 ━━━━━━━━━━━━━━━━━━━━ 224s 237ms/step - accuracy: 0.9138 - loss: 0.2678 - val_accuracy: 0.8890 - val_loss: 0.2893
Epoch 2/10
927/927 ━━━━━━━━━━━━━━━━━━━━ 256s 232ms/step - accuracy: 0.9383 - loss: 0.1620 - val_accuracy: 0.9160 - val_loss: 0.2094
Epoch 3/10
927/927 ━━━━━━━━━━━━━━━━━━━━ 212s 229ms/step - accuracy: 0.9414 - loss: 0.1457 - val_accuracy: 0.9454 - val_loss: 0.1387
Epoch 4/10
927/927 ━━━━━━━━━━━━━━━━━━━━ 271s 238ms/step - accuracy: 0.9463 - loss: 0.1339 - val_accuracy: 0.9477 - val_loss: 0.1307
Epoch 5/10
927/927 ━━━━━━━━━━━━━━━━━━━━ 255s 230ms/step - accuracy: 0.9487 - loss: 0.1243 - val_accuracy: 0.9496 - val_loss: 0.1253
Epoch 6/10
927/927 ━━━━━━━━━━━━━━━━━━━━ 263s 231ms/step - accuracy: 0.9495 - loss: 0.1209 - val_accuracy: 0.9510 - val_loss: 0.1199
Epoch 7/10
927/927 ━━━━━━━━━━━━━━━━━━━━ 265s 235ms/step - accuracy: 0.9505 - loss: 0.1205 - val_accuracy: 0.9501 - val_loss: 0.1317
Epoch 8/10
927/927 ━━━━━━━━━━━━━━━━━━━━ 262s 235ms/step - accuracy: 0.9521 -

In [10]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 예측
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(int)

# 평가 출력
print(classification_report(y_test, y_pred_class))
print(confusion_matrix(y_test, y_pred_class))


1854/1854 ━━━━━━━━━━━━━━━━━━━━ 39s 21ms/step
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     53233
           1       0.96      0.57      0.71      6069

    accuracy                           0.95     59302
   macro avg       0.95      0.78      0.84     59302
weighted avg       0.95      0.95      0.95     59302

[[53077   156]
 [ 2623  3446]]


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

# 🔹 데이터 로드

# 🔹 라벨 부여 (0~4: 각각의 DDoS 기법)
labels_ddos = np.array(
    [0]*len(http) + [1]*len(icmp) + [2]*len(tcp_ack) + [3]*len(tcp_other) + [4]*len(udp_other)
)

# 🔹 입력 데이터 합치기
X_2 = np.concatenate([http, icmp, tcp_ack, tcp_other, udp_other], axis=0)
X_2 = X_2.reshape((X_2.shape[0], X_2.shape[1], 1))  # (샘플 수, 1024, 1)

# 🔹 학습 / 테스트 분리
X2_train, X2_test, y2_train, y2_test = train_test_split(X_2, labels_ddos, test_size=0.5, random_state=42)


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# 🔹 1D-CNN 기반 다중 분류 모델
model_2 = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(1024, 1)),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.3),

    Conv1D(64, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.3),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')  # 다중 클래스 출력
])

model_2.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 🔹 학습
model_2.fit(X2_train, y2_train, epochs=10, batch_size=64, validation_data=(X2_test, y2_test))


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


95/95 ━━━━━━━━━━━━━━━━━━━━ 26s 248ms/step - accuracy: 0.4519 - loss: 2.1140 - val_accuracy: 0.5071 - val_loss: 0.7923
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 41s 249ms/step - accuracy: 0.5228 - loss: 0.7330 - val_accuracy: 0.5222 - val_loss: 0.7413
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 41s 249ms/step - accuracy: 0.5018 - loss: 0.7119 - val_accuracy: 0.5039 - val_loss: 0.7429
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 47s 318ms/step - accuracy: 0.5263 - loss: 0.7058 - val_accuracy: 0.5119 - val_loss: 0.7355
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 35s 252ms/step - accuracy: 0.5153 - loss: 0.7014 - val_accuracy: 0.5149 - val_loss: 0.7069
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 41s 250ms/step - accuracy: 0.5181 - loss: 0.6958 - val_accuracy: 0.5091 - val_loss: 0.7280
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 26s 276ms/step - accuracy: 0.5338 - loss: 0.7043 - val_accuracy: 0.5117 - val_loss: 0.7224
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 24s 252ms/step - accuracy: 0.5127 - loss: 0.6946 - val_accuracy: 0.511

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 🔹 예측
y2_pred_probs = model_2.predict(X2_test)
y2_pred_classes = np.argmax(y2_pred_probs, axis=1)

# 🔹 클래스 이름
attack_labels = ['HTTP Flood', 'ICMP Flood', 'TCP ACK Flood', 'TCP 기타', 'UDP 기타']

# 🔹 텍스트 리포트
print("📋 분류 성능 리포트:")
print(classification_report(y2_test, y2_pred_classes, target_names=attack_labels))

# 🔹 혼동 행렬 시각화
cm = confusion_matrix(y2_test, y2_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=attack_labels)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title("📊 DDoS 공격 기법별 분류 혼동 행렬")
plt.show()
